In [ ]:
import os
import requests

# Get PDF document path
pdf_path = "KAN.pdf"

In [3]:
import fitz
from tqdm.auto import tqdm

pdf_path = "KAN.pdf"

def text_formatter(text: str) -> str: 
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()

    # Potentially more text formatting functions can go here
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = [] 
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({"page_number": page_number+1,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_setence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4, # 1 token = ~4 characters
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': 1,
  'page_char_count': 2287,
  'page_word_count': 347,
  'page_setence_count_raw': 14,
  'page_token_count': 571.75,
  'text': 'KAN: Kolmogorov–Arnold Networks Ziming Liu1,4∗ Yixuan Wang2 Sachin Vaidya1 Fabian Ruehle3,4 James Halverson3,4 Marin Soljaˇci´c1,4 Thomas Y. Hou2 Max Tegmark1,4 1 Massachusetts Institute of Technology 2 California Institute of Technology 3 Northeastern University 4 The NSF Institute for Artificial Intelligence and Fundamental Interactions Abstract Inspired by the Kolmogorov-Arnold representation theorem, we propose Kolmogorov- Arnold Networks (KANs) as promising alternatives to Multi-Layer Perceptrons (MLPs). While MLPs have fixed activation functions on nodes (“neurons”), KANs have learnable activation functions on edges (“weights”). KANs have no linear weights at all – every weight parameter is replaced by a univariate function parametrized as a spline. We show that this seemingly simple change makes KANs outperform MLPs in terms of accurac

In [4]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()


,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count,text
0,1,2287,347,14,571.75,"KAN: Kolmogorov–Arnold Networks Ziming Liu1,4∗..."
1,2,3854,571,25,963.50,1 Introduction Multi-layer perceptrons (MLPs) ...
2,3,2916,437,21,729.00,Figure 2.1: Our proposed Kolmogorov-Arnold net...
3,4,2735,452,23,683.75,Figure 2.2: Left: Notations of activations tha...
4,5,2754,535,22,688.50,"As mentioned, such a network is known to be to..."


In [5]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)

    # Make sure all sentences are strings (the default type is a spaCy datatype)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    # Count the sentences
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/50 [00:00<?, ?it/s]

In [9]:
import random
random.sample(pages_and_texts, k=1)

[{'page_number': 7,
  'page_char_count': 2829,
  'page_word_count': 550,
  'page_setence_count_raw': 24,
  'page_token_count': 707.25,
  'text': '(2) with layers of equal width n0 = n1 = · · · = nL = N, (3) with each spline of order k (usually k = 3) on G intervals (for G + 1 grid points). Then there are in total O(N 2L(G + k)) ∼ O(N 2LG) parameters. In contrast, an MLP with depth L and width N only needs O(N 2L) parameters, which appears to be more efficient than KAN. For- tunately, KANs usually require much smaller N than MLPs, which not only saves parameters, but also achieves better generalization (see e.g., Figure 3.1 and 3.3) and facilitates interpretability. We remark that for 1D problems, we can take N = L = 1 and the KAN network in our implementation is nothing but a spline approximation. For higher dimensions, we characterize the generalization behavior of KANs with a theorem below. 2.3 KAN’s Approximation Abilities and Scaling Laws Recall that in Eq. (2.1), the 2-Layer width

In [8]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count,page_sentence_count_spacy
count,50.00,50.00,50.00,50.00,50.00,50.00
mean,25.50,2692.48,446.52,24.72,673.12,24.02
std,14.58,1242.93,228.45,14.89,310.73,14.47
min,1.00,32.00,6.00,1.00,8.00,1.00
25%,13.25,2184.25,369.50,19.00,546.06,19.00
50%,25.50,2886.00,444.50,23.00,721.50,23.00
75%,37.75,3346.00,567.75,28.00,836.50,28.00
max,50.00,6578.00,1302.00,64.00,1644.50,61.00


In [10]:
num_chunk_list_size = 10

def chunk_text(input_text: list[str], chunk_size: int= num_chunk_list_size) -> list[list[str]]:
    return [input_text[i: i+chunk_size] for i in range(0, len(input_text), chunk_size)]

In [11]:
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = chunk_text(item["sentences"])
    item["sentence_chunks_num"] = len(item["sentence_chunks"])

  0%|          | 0/50 [00:00<?, ?it/s]

In [18]:
rnd = random.sample(pages_and_texts, k=1)
rnd_chunk = rnd[0]["sentence_chunks"]
rnd_chunk


[['Figure C.1: Effects of hyperparameters on interpretability results.',
  'E Remark on grid size For both PDE and regression tasks, when we choose the training data on uniform grids, we witness a sudden increase in training loss (i.e., sudden drop in performance) when the grid size is updated to a large level, comparable to the different training points in one spatial direction.',
  'This could be due to implementation of B-spline in higher dimensions and needs further investigation.',
  'F KANs for special functions We include more results on the special function dataset (Section 3.2).',
  'Figure F.2 and F.1 visualize minimal KANs (under the constraint test RMSE < 10−2) and best KANs (with the lowest test RMSE loss) for each special function fitting task.',
  '45']]